In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install segment-geospatial

In [ ]:
!conda install -y -c conda-forge gdal

In [ ]:
import os
from xml.etree import ElementTree as ET
from samgeo import tms_to_geotiff


def get_bbox_from_kml(kml_file):
    """
    Extract the bounding box from a KML file.
    """
    try:
        tree = ET.parse(kml_file)
        root = tree.getroot()

        # Find the <coordinates> tag (assumes one polygon in the KML)
        namespace = {"kml": "http://www.opengis.net/kml/2.2"}
        coords = root.find(".//kml:coordinates", namespace)

        if coords is None:
            raise ValueError(f"No coordinates found in KML: {kml_file}")

        # Parse coordinates into a list of (longitude, latitude)
        points = []
        for coord in coords.text.strip().split():
            lon, lat, _ = map(float, coord.split(","))
            points.append((lon, lat))

        # Calculate bounding box
        lons, lats = zip(*points)
        bbox = [min(lons), min(lats), max(lons), max(lats)]
        return bbox

    except Exception as e:
        raise ValueError(f"Error processing KML file {kml_file}: {e}")


if __name__ == "__main__":
    kml_folder = r"/kaggle/input/copper-shape"  # Folder containing KML files
    output_folder = r"/kaggle/working/Copper"  # Output folder for GeoTIFFs
    zoom = 18
    source = "Satellite"

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each KML file
    for entry in os.scandir(kml_folder):
        if entry.is_file() and entry.name.endswith(".kml"):
            base_filename, _ = os.path.splitext(entry.name)
            output_filename = f"{base_filename}.tif"
            kml_path = entry.path
            output_path = os.path.join(output_folder, output_filename)

            try:
                bbox = get_bbox_from_kml(kml_path)
                print(f"Processing file: {entry.name}")
                print(f"BBox: {bbox}")

                # Convert TMS to GeoTIFF
                tms_to_geotiff(output=output_path, bbox=bbox, zoom=zoom, source=source, overwrite=True)
                print(f"Successfully processed: {entry.name}")

            except Exception as e:
                print(f"Failed to process {entry.name}: {e}")


In [ ]:
import shutil
shutil.make_archive('working', 'zip', '/kaggle/working/Copper')